## LLM Inference on P3.2xlarge EC2 Instance
This EC2 instance has 1 NVIDIA Tesla V100 GPU. We will run inference on the mistralai/Mistral-7B-v0.1 model using GPU

## Install necessary packages

In [1]:
!pip install -U -q accelerate
!pip install transformers
!pip install huggingface_hub


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Log into Huggingface

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Load the model and Tokenizer from Huggingface

In [3]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mistral-7B-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

model-00002-of-00002.safetensors:  59%|#####9    | 2.68G/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

## Run the inference on GPU

In [4]:
import time

text = "Write be a 150 word essay Why is health important to everyone?"
device = "cuda"

inputs = tokenizer(text, return_tensors="pt").to(device)

# Get start time
t1 = time.time()

outputs = model.generate(**inputs, max_new_tokens=300)

# Get end time
t2 = time.time()

# Get total time taken
t3 = t2 - t1

response = (tokenizer.decode(outputs[0], skip_special_tokens=True))
print(response)

# Calculate the number of output tokens
tokens = tokenizer.tokenize(response)
num_tokens = (len(tokens))
print("Number of tokens generated: ", num_tokens)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write be a 150 word essay Why is health important to everyone?

Health is important to everyone because it is the foundation of a happy and productive life. Without good health, it is difficult to enjoy life and achieve our goals. Good health allows us to be active, productive, and engaged in our communities. It also helps us to avoid illness and injury, which can be costly and disruptive to our lives.

Good health is also essential for our mental and emotional well-being. When we are healthy, we are better able to cope with stress and handle the challenges of life. We are also more likely to have positive relationships with others and to feel a sense of purpose and meaning in our lives.

In addition, good health is important for our economic well-being. When we are healthy, we are more likely to be productive and to contribute to the economy. We are also less likely to need costly medical care, which can be a burden on our finances.

Overall, good health is essential for a happy and p

## Calculate Throughput and total time

In [5]:
# Print total time taken
print(t3,": seconds")

# Calculate tokens per secon
tokens_per_second = num_tokens/t3

print("Number of Tokens per second: ", tokens_per_second)

53.34893536567688 : seconds
Number of Tokens per second:  4.686129128658162
